In [182]:
# import libraries
import pysam
import pprint

In [183]:
# create an alignment file for "merged-tumor.bam", located in /sbgenomics/project-files/
alignmentFile = pysam.AlignmentFile("/sbgenomics/project-files/merged-tumor.bam", "rb")

In [184]:
# initialize variables for later
firstRead = {}
fields = ['query_name', 'flag', 'mapping_quality', 'cigartuples', 'query_length', 'query_sequence', 'reference_name',
          'reference_start', 'next_reference_name', 'next_reference_start', 'query_qualities', 'tags']
numberOfFirstFlagsToCheck = 10
flags = []
numberOfReads = 0
numberOfUnmappedReads = 0
numberOfZeroMappingQualityReads = 0
numberOfNonZeroMappingQualityReads = 0
totalMappingQuality = 0
totalNonZeroMappingQuality = 0
averageMappingQuality = 0
averageNonZeroMappingQuality = 0
firstFlagAnalysis = "1187 = 0x4A3\n\
0x4A3 = 0x400 + 0x080 + 0x020 + 0x002 + 0x001\n\
0x001 => Segment is paired in sequencing\n\
0x002 => Segment is mapped in a proper pair - normally inferred during alignment\n\
0x020 => Strand of the mate\n\
0x080 => Segment is the second read in a pair\n\
0x400 => Segment is either a PCR duplicate or an optical duplicate\n\n"

In [185]:
# iterating over reads
for read in alignmentFile:
    # updating (total) number of reads and a sum for mapping quality of all reads
    numberOfReads += 1
    totalMappingQuality += read.mapping_quality
    
    # if it is a first read, inspect aligned segment fields
    if numberOfReads == 1:
        print("Flag of the first read: " + str(read.flag) + "\n" + firstFlagAnalysis + 
              "AlignedSegment fields and values of the first read:")
        for ind in range(len(fields)):
            firstRead[fields[ind]] = getattr(read, fields[ind])
        pprint.pprint(firstRead)
    
    # check out flags for some (first numberOfFirstFlagsToCheck) reads
    if numberOfReads <= numberOfFirstFlagsToCheck:
        flags.append(read.flag)
    
    # updating number of unmapped reads in the file
    if read.is_unmapped:
        numberOfUnmappedReads += 1
        
    # updating number of reads with zero quality mapping
    if read.mapping_quality == 0:
        numberOfZeroMappingQualityReads += 1
    # updating number of reads and a sum for a non-zero quality mapping reads
    else:    
        numberOfNonZeroMappingQualityReads += 1
        totalNonZeroMappingQuality += read.mapping_quality

alignmentFile.close()
        
# calculate averages
averageMappingQuality = totalMappingQuality / numberOfReads
averageNonZeroMappingQuality = totalNonZeroMappingQuality / numberOfNonZeroMappingQualityReads

Flag of the first read: 1187
1187 = 0x4A3
0x4A3 = 0x400 + 0x080 + 0x020 + 0x002 + 0x001
0x001 => Segment is paired in sequencing
0x002 => Segment is mapped in a proper pair - normally inferred during alignment
0x020 => Strand of the mate
0x080 => Segment is the second read in a pair
0x400 => Segment is either a PCR duplicate or an optical duplicate

AlignedSegment fields and values of the first read:
{'cigartuples': [(0, 76)],
 'flag': 1187,
 'mapping_quality': 27,
 'next_reference_name': '21',
 'next_reference_start': 9483381,
 'query_length': 76,
 'query_name': 'C0HVYACXX120402:7:1207:5722:57044',
 'query_qualities': array('B', [28, 28, 27, 29, 31, 30, 31, 31, 29, 31, 35, 30, 29, 31, 34, 30, 29, 23, 41, 32, 20, 30, 29, 34, 34, 29, 30, 31, 30, 30, 30, 33, 33, 26, 39, 12, 25, 19, 32, 30, 35, 28, 35, 33, 23, 33, 35, 36, 30, 38, 33, 41, 34, 35, 31, 33, 23, 30, 30, 36, 27, 32, 29, 34, 35, 41, 33, 31, 33, 29, 32, 32, 31, 31, 31, 34]),
 'query_sequence': 'TTTTCAAACAGTATCTATGCCTGCCAAATGTGAAC

In [186]:
print("Flags of the first " + str(numberOfFirstFlagsToCheck) + " reads: ")
for flag in flags:
    print(flag)

Flags of the first 10 reads: 
1187
163
99
99
99
163
99
1123
99
1123


In [187]:
print("Number of unmapped reads in the file: " + str(numberOfUnmappedReads))
print("Total number of reads in the file: " + str(numberOfReads))
print("Number of reads with zero mapping quality: " + str(numberOfZeroMappingQualityReads))
print("Average mapping quality: {}".format(averageMappingQuality))
print("Average non-zero mapping quality: {}".format(averageNonZeroMappingQuality))

Number of unmapped reads in the file: 17765
Total number of reads in the file: 2921629
Number of reads with zero mapping quality: 126628
Average mapping quality: 55.91379158681681
Average non-zero mapping quality: 58.446975510921106
